### __Introducción__

SQLAlchemy es una biblioteca de Python muy potente y flexible que facilita la interacción con bases de datos SQL. Combina la potencia de SQL con las ventajas del lenguaje de programación Python, permitiendo a los desarrolladores trabajar con bases de datos de una manera más sencilla y estructurada.

SQLAlchemy se compone de dos partes principales.

- _- Core (Núcleo)._ Proporciona un mapeo de bajo nivel y permite ejecutar sentencias SQL directamente.
- _- ORM (Object-relational mapping)._ Ofrece una abstracción de alto nivel que permite interactuar con la base de datos usando objetos Python, mapeando las tablas y filas SQL a clases y objetos de Python.

__Componentes clave.__

- _Engine_. Es el punto de conexión entre la aplicación y la base de datos. Maneja la conexión y la comunicación con la base de datos.
- _Session_. Es una interfaz de alto nivel para interactuar con la base de datos usando el ORM. Gestiona las operaciones de persistencia de los objetos.
- _Declarative base_. Es una forma de definir las tablas de la base de datos como clases Python usando el ORM.

In [66]:
from sqlalchemy import create_engine, text, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

En SQLAlchemy, no interactúas directamente con archivos .mdf (archivos de datos de SQL Server) o .ldf (archivos de registro de SQL Server). En su lugar, SQLAlchemy actúa a través de un motor de base de datos mediante una conexión a una base de datos SQL Server, usando una URL de conexión que le dice a SQLAlchemy cómo conectarse a la base de datos.

Asegurate de tener instalada esta librería.

_pip install pyodbc_

##### Crear un engine

El __engine__ es el componente principal que maneja la conexión a la base de datos. Actúa como una interfaz entre tu aplicación y la base de datos. Es responsable de establecer y gestionar la conexión, y proporciona la capacidad para ejecutar consultas SQL y otras operaciones de base de datos.

No realiza las operaciones directamente, sino que proporciona un punto de entrada para obtener conexiones y ejecutar consultas.

Se usa una URL de conexión que especifica la ubicación y las credenciales necesarias para conectar a la base de datos. SQLAlchemy se encarga de gestionar las conexiones y las operaciones en la base de datos a través de esta conexión, pero no a nivel de archivo.

In [67]:
from sqlalchemy import create_engine

# Crear un engine que se conecte a una base de datos.
Engine = create_engine('mssql+pyodbc://COMPU-02\\ARONIUM/DB?driver=ODBC+Driver+17+for+SQL+Server;Trusted_Connection=yes')

##### Crear una sesión

La __session__ es un componente que maneja la comunicación entre tu aplicación y la base de datos a nivel de transacciones. Se utiliza para realizar operaciones como añadir, eliminar o actualizar objetos en la base de datos.

O sea, está orientada a los objetos de la base de datos: gestiona el ciclo de vida de las transacciones, lo que incluye agregar objetos, realizar consultas y confirmar los cambios en la base de datos mediante commit.

In [68]:
from sqlalchemy.orm import sessionmaker

In [69]:
# Crear una clase Session con el Engine.
Session = sessionmaker(bind=Engine)

Los cambios que se realizan con la sesión son temporales hasta que se confirmen explícitamente con session.commit(). Hasta ese momento, los cambios están en memoria y no afectan la base de datos.

##### Definir una tabla y su esquema

Para definir una tabla, se utiliza el ORM de SQLAlchemy con una clase.

In [70]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

En SQLAlchemy, existe una clase (_declarative_base_) que se utiliza para definir la estructura de las tablas en la base de datos. 

Luego, creo una instancia que hereda sus métodos y atributos: __Table__. Table va a terminar siendo la clase que moldea todas las tablas que vaya creando en el documento, que van a ser instancias (aunque también van a ser clases).

In [71]:
# Base para la declaración de clases.
Table = declarative_base()

# Tabla es una instancia de la clase 'declarative_base'. 
# Lo que vamos a hacer después son objetos derivados de la clase Table.
# Table es el molde con el que se arman las tablas.

C:\Users\tomas\AppData\Local\Temp\ipykernel_16608\3319955963.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Table = declarative_base()


Definís una clase hija de Table, que es una tabla, definiendo sus columnas y tipos de datos.

In [72]:
class Usuario(Table):
    __tablename__ = 'Usuarios'        
    ID = Column(Integer, primary_key=True)
    Nombre = Column(String)
    Edad = Column(Integer)

##### Insertar el esquema de las tablas en la base de datos

In [73]:
Table.metadata.create_all(Engine)

DBAPIError: (pyodbc.Error) ('IM012', '[IM012] [Microsoft][ODBC Driver Manager] DRIVER keyword syntax error (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

Se crea una tabla, definiendo su nombre (_tablename_) y sus columnas: 
- _ID_ (se especifica que es una clave primaria con _primary key = True_ y que es de tipo numérica por el _Integer_), 
- _Nombre_ (se especifica que es de tipo string), y 
- _Edad_ (de tipo numérico).

##### Crear una tupla

In [47]:
# Crear objeto de la clase User, es decir, una fila de la tabla 'Usuarios'.
Patricio = Usuario(Nombre='Patricio', Edad=50)

##### Añadir una fila o tupla a una tabla

In [48]:
# Añadir objeto a la Sesion.
Tabla_Usuarios.add(Patricio)

### __Consultar datos de una database__

##### Extraer los nombres de las tablas

- __Engine.connect()__. Abre una conexión al motor de base de datos definido por Engine. El motor es la instancia que gestiona la conexión con la base de datos.
- __with ... as Conexion__. Utiliza un contexto (with statement) para asegurar que la conexión se maneje correctamente. Cuando salgas del contexto, la conexión se cerrará automáticamente, incluso si ocurre un error durante su uso.
- __Conexion.execute(...)__. Ejecuta una consulta SQL en la base de datos. En este caso, la consulta es "SELECT name FROM sqlite_master WHERE type='table';".
- __sqlite_master__. Es una tabla interna en SQLite que contiene información sobre las tablas, índices y otros objetos en la base de datos.
- __type='table'__. Filtra los resultados para que solo obtengas las tablas (y no índices u otros objetos).
- __fetchall()__. Recupera todos los resultados de la consulta y los devuelve como una lista de tuplas. Cada tupla contiene un solo valor: el nombre de una tabla.

In [49]:
# Crear una conexión al motor.
with Engine.connect() as Conexion:
    # Obtener los nombres de las tablas
    Nombres_Tablas = Conexion.execute(text("SELECT name FROM sqlite_master WHERE type='table';")).fetchall()
    print(Nombres_Tablas)

[('Usuarios',)]


##### Extraer todos los datos de columnas específicas

In [75]:
Todos_Los_Usuarios = Tabla_Usuarios.query(Usuario).all()

for Fila in Todos_Los_Usuarios:
    print(Fila.Nombre, Fila.Edad)

Patricio 50


##### Subtítulo

Texto

In [51]:
# Código

### __Tablas (Clases)__

##### Subtítulo

Texto

In [52]:
# Código

##### Subtítulo

Texto

In [53]:
# Código

### __Consultar datos__

##### Subtítulo

Texto

In [54]:
# Código

##### Subtítulo

Texto

In [55]:
# Código

##### Subtítulo

Texto

In [56]:
# Código

### __Tuplas__

##### Subtítulo

Texto

In [57]:
# Código

##### Subtítulo

Texto

In [58]:
# Código

##### Subtítulo

Texto

In [59]:
# Código

### __Next__

##### Subtítulo

Texto

In [60]:
# Código

##### Subtítulo

Texto

In [61]:
# Código

##### Subtítulo

Texto

In [62]:
# Código

### __Itertools__

##### Subtítulo

Texto

In [63]:
# Código

##### Subtítulo

Texto

In [64]:
# Código

##### Subtítulo

Texto

In [65]:
# Código

### __Bucles for__

##### Subtítulo

Texto

In [66]:
# Código

##### Subtítulo

Texto

In [67]:
# Código

##### Subtítulo

Texto

In [68]:
# Código

### __Bucles while__

##### Subtítulo

Texto

In [69]:
# Código

##### Subtítulo

Texto

In [70]:
# Código

##### Subtítulo

Texto

In [71]:
# Código

### __Generadores: yield__

##### Subtítulo

Texto

In [72]:
# Código

##### Subtítulo

Texto

In [73]:
# Código

##### Subtítulo

Texto

In [74]:
# Código